![logo](./img/LogoLine_horizon_C3S.png)

<br>

# Seasonal forecast download and plot for one coordinate

### About

This notebook is based on "Seasonal Forecast Anomalies": https://ecmwf-projects.github.io/copernicus-training-c3s/sf-anomalies.html
It was mainly modified to use daily resolution instead of monthly.
It provides a practical introduction to calculating seasonal forecast with data from the Copernicus Climate Change Service (C3S). C3S seasonal forecast products are based on data from several state-of-the-art seasonal prediction systems. In this tutorial we shall focus on the [ECMWF SEAS5 model](https://confluence.ecmwf.int/display/CKB/Description+of+SEAS5+C3S+contribution), which is one of the forecasting systems available through C3S.

The tutorial will demonstrate how to access real-time forecast data of total precipitation, with a forecast start date in May 2023 and 6-7 monthly lead times (up to October 2023). Hindcast data for the same start date and lead-time months in the reference period 1993 to 2016 will also be downloaded. 

The tutorial will then show how to interpolate in a coordinate for both the forecast and hindcast data. (The climate mean for the reference period will be computed and this reference mean will be subtracted from the real-time forecast data to derive monthly anomalies BORRAR?). These will be visualised as time series. Finally, 3-monthly anomalies will be calculated and visualised in an interactive plot, as a demonstration of how to reproduce similar [charts available through C3S](https://climate.copernicus.eu/charts/c3s_seasonal/).


The notebook has the following outline:
* 1 - Download data from the CDS
* 2 - Hindcast data 
* 3 - Real-time forecasts

<br>

<div class="alert alert-block alert-success">
<b>NOTE</b>: <br>
    <a href="https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-postprocessed-single-levels">Precomputed anomalies are also available through the CDS</a>. Note these may be slightly different due to minor differences in the way they are computed (e.g. months of constant length, 30 days)  and also due to GRIB packing discretisation. <a href="https://confluence.ecmwf.int/display/UDOC/Why+are+there+sometimes+small+negative+precipitation+accumulations+-+ecCodes+GRIB+FAQ">See here for more detials.</a></div>

Please see here the full documentation of the [C3S Seasonal Forecast Datasets](https://confluence.ecmwf.int/display/CKB/C3S+Seasonal+Forecasts%3A+datasets+documentation). This notebook introduces you to the [seasonal forecast monthly statistics](https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-monthly-single-levels?tab=overview) datasets on single levels (as opposed to multiple levels in the atmosphere).

<style>
td, th {
   border: 1px solid white;
   border-collapse: collapse;
}
</style>
<table align="left">
  <tr>
    <th>Run the tutorial via free cloud platforms: </th>
    <th><a href="https://mybinder.org/v2/gh/ecmwf-projects/copernicus-training-c3s/main?labpath=sf-anomalies.ipynb">
        <img src = "https://mybinder.org/badge.svg" alt = "Binder"></th>
    <th><a href="https://kaggle.com/kernels/welcome?src=https://github.com/ecmwf-projects/copernicus-training-c3s/blob/main/sf-anomalies.ipynb">
        <img src = "https://kaggle.com/static/images/open-in-kaggle.svg" alt = "Kaggle"></th>
    <th><a href="https://colab.research.google.com/github/ecmwf-projects/copernicus-training-c3s/blob/main/sf-anomalies.ipynb">
        <img src = "https://colab.research.google.com/assets/colab-badge.svg" alt = "Colab"></th>
  </tr>
</table>

<hr>

### Install packages

In [ ]:
# Install CDS API for downloading data from the CDS
#!pip install cdsapi

In [ ]:
# Install cfgrib to enable us to read GRIB format files
#!conda install -c conda-forge cfgrib -y

In [1]:
# Miscellaneous operating system interfaces
import os

# CDS API
import cdsapi

# To map GRIB files to NetCDF Common Data Model
import cfgrib

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Interactive HTML widgets
import ipywidgets as widgets

# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()

ImportError: Can't determine version for bottleneck

<hr>

In [3]:
## 1. Request data from the CDS programmatically with the CDS API

The first step is to request data from the Climate Data Store programmatically with the help of the CDS API. Let us make use of the option to manually set the CDS API credentials. First, you have to define two variables: `URL` and `KEY` which build together your CDS API key. Below, you have to replace the `#########` with your personal CDS key. Please find [here](https://cds.climate.copernicus.eu/api-how-to) your personal CDS key.

In [2]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = '5596:e5a0ace8-18ac-417c-96a5-ecb447a7eb20'
variable_name = 'total_precipitation'
var_short = 'tp'
month_num = 5
forecast_year = 2023

Here we specify a data directory in which we will download our data and all output files that we will generate:

In [3]:
DATADIR = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/forecast_download/'

The next step is then to request the seasonal forecast daily data on single levels with the help of the CDS API. 
Below, we download two separate files of the variable for six monthly lead times.
For instance to start date in May: 
                                                                                                                                                             
* **Retrospective forecasts (Hindcasts) for 1993 to 2016**
* **Forecasts for May 2023**

Seasonal forecast data are disseminated in the GRIB data format. Let us store the data in the main working directory with the names: 
* `ecmwf_seas5_1993_2016_05_hindcast_daily_tp.grib` and 
* `ecmwf_seas5_2023_05_forecast_daily_tp.grib`.

Running the code block below will download the data from the CDS as specified by the following API keywords:

> **Format**: `Grib` <br>
> **Originating centre**: `ECMWF` <br>
> **System**: `5` *this refers to SEAS5* <br>
> **Variable**: `Total precipitation` or `2m temperature` <br>
> **Product type**: `Monthly mean` *all ensemble members will be retrieved* <br>
> **Year**: `1993 to 2016` *for the hindcast* `2023` *for the forecast* <br>
> **Month**: `5` *May* <br>
> **Leadtime month**: `1 to 6` *May to October*

If you have not already done so, you will need to accept the **terms & conditions** of the data before you can download it. These can be viewed and accepted in the [CDS download page](https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-monthly-single-levels?tab=form) by scrolling to the end of the download form.

<div class="alert alert-block alert-success">
<b>NOTE</b>: <br>
    The API request below can be generated automatically from the <a href="https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-monthly-single-levels?tab=form">CDS download page</a>. At the end of the download form there is a <code>Show API request</code> icon, which allows a copy-paste of the code below.</div>

In [20]:
c = cdsapi.Client(url=URL, key=KEY)

# Hindcast data request
c.retrieve(
    'seasonal-original-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ecmwf',
        'system': '5',
        'variable': variable_name,
        'year': [
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016'
        ],
        'month': month_num,
        'day': '01',
        'leadtime_hour': [
            '24', '48', '72',
            '96', '120', '144',
            '168', '192', '216',
            '240', '264', '288',
            '312', '336', '360',
            '384', '408', '432',
            '456', '480', '504',
            '528', '552', '576',
            '600', '624', '648',
            '672', '696', '720',
            '744', '768', '792',
            '816', '840', '864',
            '888', '912', '936',
            '960', '984', '1008',
            '1032', '1056', '1080',
            '1104', '1128', '1152',
            '1176', '1200', '1224',
            '1248', '1272', '1296',
            '1320', '1344', '1368',
            '1392', '1416', '1440',
            '1464', '1488', '1512',
            '1536', '1560', '1584',
            '1608', '1632', '1656',
            '1680', '1704', '1728',
            '1752', '1776', '1800',
            '1824', '1848', '1872',
            '1896', '1920', '1944',
            '1968', '1992', '2016',
            '2040', '2064', '2088',
            '2112', '2136', '2160',
            '2184', '2208', '2232',
            '2256', '2280', '2304',
            '2328', '2352', '2376',
            '2400', '2424', '2448',
            '2472', '2496', '2520',
            '2544', '2568', '2592',
            '2616', '2640', '2664',
            '2688', '2712', '2736',
            '2760', '2784', '2808',
            '2832', '2856', '2880',
            '2904', '2928', '2952',
            '2976', '3000', '3024',
            '3048', '3072', '3096',
            '3120', '3144', '3168',
            '3192', '3216', '3240',
            '3264', '3288', '3312',
            '3336', '3360', '3384',
            '3408', '3432', '3456',
            '3480', '3504', '3528',
            '3552', '3576', '3600',
            '3624', '3648', '3672',
            '3696', '3720', '3744',
            '3768', '3792', '3816',
            '3840', '3864', '3888',
            '3912', '3936', '3960',
            '3984', '4008', '4032',
            '4056', '4080', '4104',
            '4128', '4152', '4176',
            '4200', '4224', '4248',
            '4272', '4296', '4320',
            '4344', '4368', '4392',
            '4416', '4440', '4464',
            '4488', '4512', '4536',
            '4560', '4584', '4608',
            '4632', '4656', '4680',
            '4704', '4728', '4752',
            '4776', '4800', '4824',
            '4848', '4872', '4896',
            '4920', '4944', '4968',
            '4992', '5016', '5040',
            '5064', '5088', '5112',
            '5136', '5160',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_seas5_1993-2016_'+ str(month_num) + '_hindcast_daily_' + var_short + '.grib')

2024-04-18 20:18:54,890 INFO Welcome to the CDS
2024-04-18 20:18:54,891 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2024-04-18 20:18:55,003 INFO Request is completed
2024-04-18 20:18:55,005 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data1/adaptor.mars.external-1713433863.581071-17226-4-097efc6a-bd6f-46bc-8bf6-c96cea95ba4c.grib to /home/daniel/Documents/intoDBP/input/seasonal/ecmwf_seas5_1993-2016_5_hindcast_daily_tp.grib (29.5M)
2024-04-18 20:19:05,391 INFO Download rate 2.8M/s                                                                                                                                                          


Result(content_length=30960000,content_type=application/x-grib,location=https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data1/adaptor.mars.external-1713433863.581071-17226-4-097efc6a-bd6f-46bc-8bf6-c96cea95ba4c.grib)

In [21]:
# Forecast data request
c.retrieve(
    'seasonal-original-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ecmwf',
        'system': '51',
        'variable': variable_name,
        'year': '2023',
        'month': month_num,
        'day': '01',
        'leadtime_hour': [
            '24', '48', '72',
            '96', '120', '144',
            '168', '192', '216',
            '240', '264', '288',
            '312', '336', '360',
            '384', '408', '432',
            '456', '480', '504',
            '528', '552', '576',
            '600', '624', '648',
            '672', '696', '720',
            '744', '768', '792',
            '816', '840', '864',
            '888', '912', '936',
            '960', '984', '1008',
            '1032', '1056', '1080',
            '1104', '1128', '1152',
            '1176', '1200', '1224',
            '1248', '1272', '1296',
            '1320', '1344', '1368',
            '1392', '1416', '1440',
            '1464', '1488', '1512',
            '1536', '1560', '1584',
            '1608', '1632', '1656',
            '1680', '1704', '1728',
            '1752', '1776', '1800',
            '1824', '1848', '1872',
            '1896', '1920', '1944',
            '1968', '1992', '2016',
            '2040', '2064', '2088',
            '2112', '2136', '2160',
            '2184', '2208', '2232',
            '2256', '2280', '2304',
            '2328', '2352', '2376',
            '2400', '2424', '2448',
            '2472', '2496', '2520',
            '2544', '2568', '2592',
            '2616', '2640', '2664',
            '2688', '2712', '2736',
            '2760', '2784', '2808',
            '2832', '2856', '2880',
            '2904', '2928', '2952',
            '2976', '3000', '3024',
            '3048', '3072', '3096',
            '3120', '3144', '3168',
            '3192', '3216', '3240',
            '3264', '3288', '3312',
            '3336', '3360', '3384',
            '3408', '3432', '3456',
            '3480', '3504', '3528',
            '3552', '3576', '3600',
            '3624', '3648', '3672',
            '3696', '3720', '3744',
            '3768', '3792', '3816',
            '3840', '3864', '3888',
            '3912', '3936', '3960',
            '3984', '4008', '4032',
            '4056', '4080', '4104',
            '4128', '4152', '4176',
            '4200', '4224', '4248',
            '4272', '4296', '4320',
            '4344', '4368', '4392',
            '4416', '4440', '4464',
            '4488', '4512', '4536',
            '4560', '4584', '4608',
            '4632', '4656', '4680',
            '4704', '4728', '4752',
            '4776', '4800', '4824',
            '4848', '4872', '4896',
            '4920', '4944', '4968',
            '4992', '5016', '5040',
            '5064', '5088', '5112',
            '5136', '5160',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_seas5_'+ str(forecast_year) + '_' + str(month_num) + '_forecast_daily_' + var_short + '.grib')

2024-04-18 20:21:12,317 INFO Welcome to the CDS
2024-04-18 20:21:12,318 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2024-04-18 20:21:12,628 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data7/adaptor.mars.external-1713437280.8940856-18060-7-ab119f0e-2b0f-40f6-bcff-6338b54c7ec2.grib to /home/daniel/Documents/intoDBP/input/seasonal/ecmwf_seas5_2023_5_forecast_daily_tp.grib (2.5M)
2024-04-18 20:21:13,590 INFO Download rate 2.6M/s                                                                                                                                                          


Result(content_length=2631600,content_type=application/x-grib,location=https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data7/adaptor.mars.external-1713437280.8940856-18060-7-ab119f0e-2b0f-40f6-bcff-6338b54c7ec2.grib)

## 2. Seasonal hindcast climate 

Seasonal forecasts are affected by systematic errors (biases) which are dependent on the leadtime, the time of year, the variable and the location. Hindcast data can help us to understand and account for these biases. We will see the hindcast climate for each lead time month, over the years 1993 to 2016.

#### Read the downloaded data

We will use the Python library [xarray](http://xarray.pydata.org/en/stable/) and its function `open_dataset` to read the GRIB file of the hindcast data, specifying the keyword argument `engine` and `'cfgrib'`. [cfgrib](https://github.com/ecmwf/cfgrib) is a Python interface to map GRIB files to the NetCDF Common Data Model using [ecCodes](https://github.com/ecmwf/eccodes).

The result is a `xarray.Dataset` object with five dimensions:

> **Number**: Ensemble members (25) <br>
> **Time**: Forecast start date for each year (1st of May) <br>
> **Step**: Lead time (nanoseconds in each leadtime month) <br>
> **Latitude**: Latitudes in 1 deg resolution<br>
> **Longitude**: Longitudes in 1 deg resolution and in 0-360 grid<br>

In [10]:
hind = xr.open_dataset(f'{DATADIR}/ecmwf_seas5_1993-2016_'+ str(month_num) + '_hindcast_daily_' + var_short + '.grib', engine='cfgrib')

#### Extract data array from dataset

The `xarray.Dataset` object into which we have read the data from the downloaded GRIB files may contain arrays of multiple variables (even if we have only one: total precipitation). Another xarray data structure, `xarray.DataArray`, facilitates operations on single variables. We will use this to further process our data. You can select the relevant DataArray from a Dataset by specifying the name of the variable (in our case `tprate`) in square brackets `[]`. 

In [11]:
hind_var = hind[var_short]

In [12]:
# Define the coordinates to interpolate to
interp_latitude = 41.5
interp_longitude = 2.5

# Interpolate the values for the specified coordinates
hind_interp = hind_var.interp(latitude=interp_latitude, longitude=interp_longitude, method="nearest")

In [33]:
year_subset = '2009-05-01T00:00:00.000000000'
subset_hind = hind_interp.sel(time=year_subset)

Total precipitation data is accumulated in time, to get the daily value the np.diff() is applied

In [34]:
num_members = len(subset_hind)
subset_hind_diff = subset_hind.diff(dim='step') # by default does it along the last axis 